In [1]:
import pandas as pd
import numpy as np
import glob
import os
from scipy import stats
import tensorflow as tf
import math
import random
import matplotlib.pyplot as plt

fps = 120
moving_avg_len = 10
filter_value = 20
length = 200
threshold_pattern=0.4
#10.18. 목



def readFileData(file):
    column_names = ['wmx1', 'wmy1', 'wmz1']

    data = pd.read_csv(file, skiprows = 1 , names = column_names)
 
    wx = data["wmx1"]
    wy = data["wmy1"]
    wz = data["wmz1"]
    
    return np.dstack([wx,wy,wz])[0]

def readData(directory):
    records = []
    labels = []
    gesture_num_records = []
    allFiles = glob.glob("*.csv")
    for i,file in enumerate(allFiles):
        fileName = os.path.basename(file)
        print("file name is", fileName,"num is",i)
        (name, ext) = os.path.splitext(fileName)
        print("name is ",name,"ext is",ext)
        parts = name.split("_")
        if (True):
            label = parts[1] #'circle', 'triangle', 'X'
            fileData = readFileData(file)
            print("fileData :",len(fileData[1]))
            records.append(fileData)
            #print("file data is ",fileData)
            labels.append(label)
            gesture_num_records.append(len(fileData))

    return (records, labels, gesture_num_records)

def movingavg(x,y,z):
    x_avg = np.zeros(moving_avg_len)/moving_avg_len
    y_avg = np.zeros(moving_avg_len)/moving_avg_len
    z_avg = np.zeros(moving_avg_len)/moving_avg_len
    x_avg = np.convolve(x,x_avg,'same')
    y_avg = np.convolve(y,y_avg,'same')
    z_avg = np.convolve(z,z_avg,'same')
    return [x_avg,y_avg,z_avg]

def splitData(records):##sungshil
    
    term=fps // 24
    record_sum = []
    record = []
    record_all = []
    sum_now = 0
    sum_pre = 0
    ct = 0
    start = 0
    end = 0
    num_pattern=[]
    for t in range(len(records)):
        ct = 0
        record_one=[]
        for i in range((len(records[t])//(term))-100):
            sum = 0
            record_x = np.empty(shape=[1],dtype=float)
            record_y = np.empty(shape=[1],dtype=float)
            record_z = np.empty(shape=[1],dtype=float)
            for j in range(fps//2):
                record_x = np.append(record_x,records[t][term*i+j][0])
                record_y = np.append(record_y,records[t][term*i+j][1])
                record_z = np.append(record_z,records[t][term*i+j][2])
            for k in range(fps//2 - 2):
                sum = sum+(record_x[k]-record_x[k+1])**2+(record_y[k]-record_y[k+1])**2+(record_z[k]-record_z[k+1])**2

            sum_pre = sum_now
            sum_now = sum
            record_sum.append(sum)
            if (sum_pre < threshold_pattern and sum_now > threshold_pattern):
                start = term * i
            if (sum_pre > threshold_pattern and sum_now < threshold_pattern):
                end = term * i  
                s = []
                if (end-start)>120:
                    ct = ct + 1
                    #print(">> start : ",start)
                    #print(" >> end : ",end,", step : " ,ct, ", length of end-start : ",(end-start))
                    for k in range(end-start):
                        #t=(records[0][term*i+j+k])
                        #print("t is",t)
                        s.append(records[t][start+k])
                        #print("s is ",s)
                    record_one.append(s)
        record_all.append(record_one)
        print("number of patterm : ",ct)
        num_pattern.append(ct)
        #print("average is",np.average(record_sum))
        #plt.plot(record_sum[1500:2000])
 
    return (record_all,num_pattern)

def getRecordsMaxLength(records):
    maxLen = 0
    for record in records:
        if (len(record) > maxLen):
            maxLen = len(record)
        
    return maxLen

def shortenRecordsLen(records, length, first_label):
    ret1 = []
    new_labels = []
    for num in range(len(records)):
        ret2 = []
        for index in range(len(records[num])):
            s = []
            if (len(records[num][index])>length):
                record = records[num][index]
                if len(record) < 1000:
                    new_labels.append(first_label[num])
                    for i in range(length):
                        ret3=[]                      
                        t=i*(len(record)/length)
                        if t%1 == 0:
                            s.append(record[int(t)])
                        else:
                            a = t//1
                            s1 = t-a
                            s2 = a+1-t
                            for j in range(3):
                                ret3.append( s1*record[int(a)][j] + s2*record[int(a)][j] )
                            s.append(ret3)
                ret2.append(s)           
                
        ret1.append(ret2)
    return ret1, new_labels

def labeling(records, num_pattern):
    labels = []
    '''recods 합쳐진 다음에 num_pattern로 제스쳐 개수 계산하지 말고 records length로 바꾸기 '''
    for i in range(num_pattern[0] + num_pattern[1] + num_pattern[2]):
        if i < num_pattern[0] :
            labels.append('circle')
        elif i < num_pattern[0] + num_pattern[1]:
            labels.append('triangle')
        else :labels.append('X')
    print("labelling - labels length : ", len(labels))
    return labels

def pattern_reshape(records,length,labels):
    segment=[]
    record_po=np.empty(0)
    record_lab=[]
    ct=0
    for i in range(len(records)):
        for j in range(len(records[i])): # num of gesture 횟수만큼
            length_ary=[]
            for k in range(len(records[i][j])): # 150회만큼
                x=np.array(records[i][j][k][0]) # files X num of gesture X 150 X 3
                y=np.array(records[i][j][k][1])
                z=np.array(records[i][j][k][2])
                length_ary.append([x,y,z])
            if len(records[i][j])==length:
                segment.append([length_ary,labels[ct]])
                ct+=1

    for k in range(len(segment)):
        record_lab.append(segment[k][1])
        for t in range(len(segment[k][0])): # x,y,z
            record_po=np.append(record_po,segment[k][0][t])
            
            
    return record_po,record_lab

    
def missing_noise(records):
    for i in range(len(records)):
        for j in range(len(records[i])):
            for t in range(len(records[i][j])):
                records[i][j][t]=random_zero(records[i][j][t])
    return records

def random_zero(records):
    a=random.random()<0.10
    if a:
        return np.array([0,0,0])
    return records


    
            
def weight_variable(shape):            #가중치 생성
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):                         #편향 생성
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def depthwise_conv2d(x, W):
    return tf.nn.depthwise_conv2d(x,W, [1, 1, 1, 1], padding='VALID')

def apply_depthwise_conv(x,kernel_size,num_channels,depth):#각 채널마다 활성곱 수행후 ReLu활성화 함수로 pass
    weights = weight_variable([1, kernel_size, num_channels, depth])
    biases = bias_variable([depth * num_channels])
    return tf.nn.relu(tf.add(depthwise_conv2d(x, weights),biases))
    
def apply_max_pool(x,kernel_size,stride_size):#생성된 convolution층에 pooling 적용
    return tf.nn.max_pool(x, ksize=[1, 1, kernel_size, 1], 
                          strides=[1, 1, stride_size, 1], padding='VALID')

C:\Users\sskim\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
(records,labels, gesture_num_records) = readData("dr") #labels는 
records_test=records
labels_test=labels
gesture_num_records_test=gesture_num_records
print(gesture_num_records)
print("len(records) : ", len(records))
print("len(labels) : ", len(labels))
print("**len_record : ",len(records))
records, nums_pattern = splitData(records)
records_test=records
nums_pattern_test =nums_pattern

###################################################### noise
#records=missing_noise(records)
records_test=missing_noise(records_test)
#####################################################

file name is Converted_circle_Goeun_1012.csv num is 0
name is  Converted_circle_Goeun_1012 ext is .csv
fileData : 3
file name is Converted_circle_Sua_1012.csv num is 1
name is  Converted_circle_Sua_1012 ext is .csv
fileData : 3
file name is Converted_circle_Sungshil_1012.csv num is 2
name is  Converted_circle_Sungshil_1012 ext is .csv
fileData : 3
file name is Converted_triangle_Goeun_1012.csv num is 3
name is  Converted_triangle_Goeun_1012 ext is .csv
fileData : 3
file name is Converted_triangle_Sua_1012.csv num is 4
name is  Converted_triangle_Sua_1012 ext is .csv
fileData : 3
file name is Converted_triangle_Sungshil_1012.csv num is 5
name is  Converted_triangle_Sungshil_1012 ext is .csv
fileData : 3
file name is Converted_X_Goeun_1012.csv num is 6
name is  Converted_X_Goeun_1012 ext is .csv
fileData : 3
file name is Converted_X_Sua_1012.csv num is 7
name is  Converted_X_Sua_1012 ext is .csv
fileData : 3
file name is Converted_X_Sungshil_1012.csv num is 8
name is  Converted_X_Sungshi

In [3]:
len_label=len(records)
len_label_test=len(records)
shortened_records, new_labels = shortenRecordsLen(records, length, labels)
shortened_records_test, new_labels_test = shortenRecordsLen(records_test, length, labels_test)
print("new_labels_len :",len(new_labels)) 

t=0
for i in range(len(shortened_records)):
    print(len(shortened_records[i]))
    t+=len(shortened_records[i])
print("sum of labels after shortenn :",t)
#labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)

new_labels_len : 746
109
115
101
55
55
28
93
100
98
sum of labels after shortenn : 754


In [4]:
#records = np.asarray(records)
records_1,labels=pattern_reshape(shortened_records,length,new_labels)
records_1_test,labels_test=pattern_reshape(shortened_records_test,length,new_labels_test)
print("len a",len(shortened_records))
print(" >> total's is ",len(records_1)/(length*3))
print("labels is ",len(labels))
###########################################################
num_ges=int(len(records_1)/(length*3))
num_ges_test=int(len(records_1_test)/(length*3))
print(num_ges_test)
print("labels_len :",len(labels))
labels=np.asarray(pd.get_dummies(labels), dtype = np.int8)
labels_test=np.asarray(pd.get_dummies(labels_test), dtype = np.int8)

print("labels_dummies :",labels)
###########################################################
reshaped_records = records_1.reshape(num_ges,1,length, 3) #recoed를 1차원으로 reshape
reshaped_records_test = records_1.reshape(num_ges_test,1,length, 3) #recoed를 1차원으로 reshape
reshaped_label=labels.reshape(len(labels), 3)
reshaped_label_test=labels.reshape(len(labels_test), 3)



len a 9
 >> total's is  746.0
labels is  746
746
labels_len : 746
labels_dummies : [[0 1 0]
 [0 1 0]
 [0 1 0]
 ...
 [1 0 0]
 [1 0 0]
 [1 0 0]]


In [5]:
train_test_split = np.random.rand(len(reshaped_records)) < 0.60
train_x = reshaped_records[train_test_split]
train_y = labels[train_test_split]
test_x = reshaped_records[~train_test_split]
test_y = labels[~train_test_split]

##########################################
train_test_split_test = np.random.rand(len(reshaped_records_test)) < 0.60
train_x_test = reshaped_records[train_test_split_test]
train_y_test = labels[train_test_split_test]
test_x_test = reshaped_records[~train_test_split_test]
test_y_test = labels[~train_test_split_test]


In [6]:
input_height = 1
input_width = length
num_labels = 3
num_channels = 3

batch_size = 10
kernel_size = 60 #??
depth = 60
num_hidden = 1000

learning_rate = 0.0001
training_epochs = 50

total_batches = train_x.shape[0] // batch_size

In [7]:
X = tf.placeholder(tf.float32, shape=[None,input_height,input_width,num_channels])
# input data
Y = tf.placeholder(tf.float32, shape=[None,num_labels])
# output data

c = apply_depthwise_conv(X,kernel_size,num_channels,depth) #1d convolution
p = apply_max_pool(c,20,2) #1d Max-Pooling
c = apply_depthwise_conv(p,6,depth*num_channels,depth//10) #1d convolution

shape = c.get_shape().as_list()
c_flat = tf.reshape(c, [-1, shape[1] * shape[2] * shape[3]])

f_weights_l1 = weight_variable([shape[1] * shape[2] * depth * num_channels * (depth//10), num_hidden])
f_biases_l1 = bias_variable([num_hidden])
f = tf.nn.tanh(tf.add(tf.matmul(c_flat, f_weights_l1),f_biases_l1))

out_weights = weight_variable([num_hidden, num_labels])
out_biases = bias_variable([num_labels])
y_ = tf.nn.softmax(tf.matmul(f, out_weights) + out_biases)
# softmax

In [8]:
loss = -tf.reduce_sum(Y * tf.log(y_))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
with tf.Session() as session: 
    tf.global_variables_initializer().run()
    for epoch in range(training_epochs):
        cost_history = np.empty(shape=[1],dtype=float)
        for b in range(total_batches):    
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :, :, :]
            batch_y = train_y[offset:(offset + batch_size), :]
        for b in range(total_batches):    
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x_test[offset:(offset + batch_size), :, :, :]
            batch_y = train_y_test[offset:(offset + batch_size), :]
            
            _, c = session.run([optimizer, loss],feed_dict={X: batch_x, Y : batch_y})
            cost_history = np.append(cost_history,c)
        print ("Epoch: ",epoch," Training Loss: ",c," Training Accuracy: ", session.run(accuracy, feed_dict={X: train_x, Y: train_y}))
    print ("Testing Accuracy:", session.run(accuracy, feed_dict={X: test_x, Y: test_y}))

Epoch:  0  Training Loss:  0.1272044  Training Accuracy:  0.35955057
Epoch:  1  Training Loss:  0.093348555  Training Accuracy:  0.35955057
Epoch:  2  Training Loss:  0.06830351  Training Accuracy:  0.4022472
Epoch:  3  Training Loss:  0.051079765  Training Accuracy:  0.51910114
Epoch:  4  Training Loss:  0.039043516  Training Accuracy:  0.68988764
Epoch:  5  Training Loss:  0.030391963  Training Accuracy:  0.7820225
Epoch:  6  Training Loss:  0.024432834  Training Accuracy:  0.8494382
Epoch:  7  Training Loss:  0.020093013  Training Accuracy:  0.8988764
Epoch:  8  Training Loss:  0.016902387  Training Accuracy:  0.9258427
Epoch:  9  Training Loss:  0.014461381  Training Accuracy:  0.941573
Epoch:  10  Training Loss:  0.012723636  Training Accuracy:  0.9640449
Epoch:  11  Training Loss:  0.011179248  Training Accuracy:  0.96629214
Epoch:  12  Training Loss:  0.01010402  Training Accuracy:  0.9707865
Epoch:  13  Training Loss:  0.009162407  Training Accuracy:  0.9730337
Epoch:  14  Trai